
## Глава 3 Теоремы теории вероятностей

**Кандидат должен знать:**

- Закон больших чисел и усиленный закон больших чисел
- Центральная предельная теорема
- Многомерные распределения

**Материалы:**
- [Райгородский курс по тер. веру лекции 8,9,12,13](https://www.youtube.com/watch?v=n7jx2MfIeX4&list=PLthfp5exSWEr8tRK-Yf-i9aXgcFJ-O16d&index=8)
- [курс ТВ  на степике из частей 2,4](https://stepik.org/course/3089)


## Задачи с кодингом


### 1. Генерация нормальной случайной величины

**Дано:**

Пусть у нас есть случайная величина $\xi \sim U[0,1]$ . 

**Задача:**
1. На основе данной случайной величины с помощью ЦПТ осуществите моделирование  ($n = 10000$) случайной величины  $\eta \sim N(\mu, \sigma^2)$

2. Постройте график полтности распределения и сравните с теоритическим. 

3. Вычислить математического ожидания и дисперсии, сравнить их с истинными значениями.

**Примечания**

1. Для генерации реализаций $\xi$ можно восппользоваться функцией numpy.random.random()
2.  Для построения теоритического графика плотности распределения нормальной величины можно воспользоваться функцией scipy.stats.norm.ppf

### 1. Генерация нормальной случайной величины

Let $X$ be a uniformly distributed random variable, and $\eta$ be a normally distributed random variable.

By the Central Limit Theorem:
$\eta = \frac{1}{N_x} \sum_{i=1}^{N_x} X_i$
<br>
$\mu_{\eta} = \frac{1}{N_x} \mu \left( \sum_{i=1}^{N_x} X_i \right) = \frac{1}{N_x} \sum_{i=1}^{N} \mu(X_i) = \mu_X$
<br>
$\text{Var}(\eta) = \frac{1}{(N_x-1)^2} \sum_{i=1}^{N_x} \text{Var}(X_i) = \frac{N_x \cdot \text{Var}(X)}{(N_x-1)^2} \approx [\text{if } N_x > 100] \approx \frac{\text{Var}(X)}{N_x}$
<br><br><br>
$\mu_X = \mu_{\eta}$
<br>
$\text{Var}(X) = N_x \cdot \text{Var}(\eta)$


Now I need random uniform values X with mu_x and var_x

In [31]:
import math
import itertools
from tqdm import tqdm

import numpy as np
from scipy.stats import norm

import plotly.express as px
import plotly.graph_objects as go

In [80]:
# eta ~ N(mu, sigma^2)
# X ~ U[a, b]
# Z ~ U[0, 1]

mu_eta = -7
var_eta = 13  # sigma^2

N_x = 100
mu_x = mu_eta
sigma_x = N_x * var_eta

a = mu_x - np.sqrt(3 * sigma_x)
b = mu_x + np.sqrt(3 * sigma_x)

N_eta = 10000
Z = np.random.random((N_eta, N_x))
eta = (a + Z * (b - a)).mean(axis=1)

print("Ey:", eta.mean())
print("var_y:", eta.var())

Ey: -7.0298475637952365
var_y: 13.045812136303185


In [81]:
fig = px.histogram(eta, nbins=40, title='Distribution of Array', labels={'value': 'Value'}, height=700, width=500)
fig.show()

In [82]:
x = eta
x.sort()
y = norm.pdf(x)

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='darkblue', width=5), name='norm pdf'))

hist_y, hist_x = np.histogram(eta, bins="auto", density=True)
fig.add_trace(go.Bar(x=hist_x, y=hist_y, opacity=0.6, name='histogram', marker=dict(color="darkgreen")))

# Update layout
fig.update_layout(
    title="Normal Distribution",
    xaxis_title="Value",
    yaxis_title="Density",
    legend=dict(x=0.8, y=0.9),
    height=700, 
    width=500
)

fig.show()

### 2. Казино 

В казино имеется две игры:

**a. Кубики:**

Игрок платит 4&#36; и бросает два кубика. Если в сумме выпало 8 и более то игрок получает приз равный полученной сумме.

**b. Разгадай код:**
    Есть пятизначный код (на каждой позиции стоит цифра от 0 до 9).  Игрок платит 10&#36; и пытается угадать код:
    
    - Если игрок меньше двух позиций то ничего не получает 
    - Если игрок угадал ровно две позиции из 5-ти, то он получает 50
    - Если игроку гадал ровно три позиции из 5-ти, то выигрыш 200
    - Если игроку гадал ровно четыре позиции из 5-ти, то выигрыш 5000
    - Если игрок угадал весь код правильно то выигрыш 100000.
    После каждой попытка код меняется.

**Задача:**

Пусть $\xi ,  \eta $ - случайные величины равные выигрышу(проигрышу) игрока в раунде.
1. Посчитать $E\xi$ и $E\eta$
2. Смоделировать случайные величины $\xi$ и $\eta $
3. Построить графики сходимости среднего выигрыша(проигрыша) игрока к математическому ожиданию.
4. Одинаковая ли скорость сходимости  сл. величины к своему математическому ожиданию. Почему?
4. Какое бы $n$ - количество эеспериментов вы би взяли, для оценки мат. ожидания?




In [47]:
win_amount = np.array([0 if i < 8 else i for i in range(0, 13)]) - 4

possible_dice_values = [i + j for i in range(1, 7) for j in range(1, 7)]
count_values = [possible_dice_values.count(i) for i in range(2, 13)]
probs = list(map(lambda x: x / 36, count_values))

E_xi = sum([prob * value for prob, value in zip(probs, win_amount[2:])])

print("probs: ", ",  ".join([f"{i + 2}: {count_values[i]}/36" for i in range(11)]))
print("probs sum", sum(count_values) / 36)
print("E_xi", round(E_xi, 5))

probs:  2: 1/36,  3: 2/36,  4: 3/36,  5: 4/36,  6: 5/36,  7: 6/36,  8: 5/36,  9: 4/36,  10: 3/36,  11: 2/36,  12: 1/36
probs sum 1.0
E_xi -0.11111


In [62]:
dice_values = np.random.randint(1, 7, 1000000)
dice_sums = dice_values[::2] + dice_values[1::2]
xi_values = win_amount[dice_sums]

print("E", xi_values.mean())

E -0.110262


### **5) Choose N to estimate E**

In [53]:
z = 1.96
eps = 0.01
sigma = xi_values.std()

n = ((z * sigma) / eps) ** 2
n

826733.46262976

## b. Разгадай код:

In [22]:
p = 0.1
q = 0.9
guess_code_win_amounts = np.array([0, 0, 50, 200, 5000, 100000]) - 10
code_game_probs = [math.comb(5, i) * p ** i * q ** (5 - i) for i in range(6)]
E_eta = sum([prob * value for prob, value in zip(code_game_probs, guess_code_win_amounts)])

print("probs", list(map(lambda x: round(x, 5), code_game_probs)))
print("probs sum", sum(code_game_probs))
print("E_eta", round(E_eta, 5))

probs [0.59049, 0.32805, 0.0729, 0.0081, 0.00045, 1e-05]
probs sum 1.0
E_eta -1.485


In [23]:
np.random.seed(113)
N_plays = 4 * 10 ** 7

zeros_code = np.array([0, 0, 0, 0, 0])
arrange_code = np.array([1, 2, 3, 4, 5])
random_values_code = np.random.randint(0, 10, (N_plays, 5))

code_guesses = np.random.randint(0, 10, (N_plays, 5))

is_guess_zeros_code = code_guesses[:] == zeros_code
zeros_guesses = is_guess_zeros_code.sum(axis=1)
zeros_guesses_win_amounts = guess_code_win_amounts[zeros_guesses]
print("zeros E", zeros_guesses_win_amounts.mean())

is_guess_arrange_code = code_guesses[:] == arrange_code
arrange_guesses = is_guess_arrange_code.sum(axis=1)
arrange_guesses_win_amounts = guess_code_win_amounts[arrange_guesses]
print("arrange E", arrange_guesses_win_amounts.mean())

is_guess_random_code = code_guesses == random_values_code
random_guesses = is_guess_random_code.sum(axis=1)
random_guesses_win_amounts = guess_code_win_amounts[random_guesses]
print("random E", random_guesses_win_amounts.mean())

print("real E", round(E_eta, 5))

zeros E -1.49063625
arrange E -1.468605
random E -1.48721875
real E -1.485


### Случайный граф


**Условие**

Пусть G(n,p) -  случайный граф на $n$ вершинах, где $p$ - это вероятность появления ребра между любыми двумя вершинами.

Обозначим за $\xi(n,p)$ - количество треугольников (т.е. полных подграфов на трёх вершинах) в случайном графе  

**Задача 1:**

1. Найти $E\xi$
2. Реализовать моделирование случайного случайного графа и функцию нахождения количества треугольников. 
3. Рассмотрим G(3,p) - случайный граф на трёх вершинах. Реализуйте $\tau$ - случайная величина равная 1 если граф полный и 0 иначе.


**Задача 2:**

- Пусть M(k) - среднее количество треугольников после k генераций случайного графа. 
- И пусть T(k)  - среднее количество треугольников после генераций случайного графа на трёх вершинах (т.е. на каждой итерации генерируем $C_n^3$ графов на трёх вершинах).

1. Верно ли что $M(k) \rightarrow E\xi$ при $k \rightarrow \infty$ 
2. Верно ли что $T(k) \rightarrow  E\xi$ при $k \rightarrow \infty$

3. Рассмотрите два графа $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.  Построите графики  $M(k)$, $T(k)$ и $E\xi$ (константа) для заданных графов.

**Задача 3**:

Проделайте аналогичные рассуждения для $D \xi$ т.е.
- Найдите  $D \xi$  математически.
- Посчитайте дисперсию количества треугольников, k раз реализуя случайный граф.
- Посчитайте дисперсию количества треугольников, k раз генерируя  $C_n^3$ реализаций случайной величины $\tau$ .
- оцените сходимость
- Постройте графики для графов  $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.

In [194]:
def generate_graph(n_graphs, n_nodes, edge_p):
    graphs = np.zeros((n_graphs, n_nodes, n_nodes))
    
    n_edges = math.comb(n_nodes, 2)
    is_edge = np.random.random((n_graphs, n_edges)) < edge_p
    node_pairs = list(itertools.combinations([i for i in range(n_nodes)], 2))
    
    for i in range(n_graphs):
        for edge, node_pair in zip(is_edge[i], node_pairs):
            if edge:
                graphs[i][node_pair[0], node_pair[1]] = 1
                graphs[i][node_pair[1], node_pair[0]] = 1
    return graphs


def count_triangles(graphs, possible_loops):
    triangles = []
    for graph in graphs:
        n_triangles = 0
        for node_0, node_1, node_2 in possible_loops:
            if graph[node_0, node_1] and graph[node_0, node_2] and graph[node_1, node_2]:
                n_triangles += 1
        triangles.append(n_triangles)
    return triangles

### Task 2 
### M(k)

In [195]:
k = 10**5
n = 10
p = 0.5

M_k_loops = list(itertools.combinations([i for i in range(n)], 3))

In [196]:
random_graphs = generate_graph(k, n, p)
n_triangles_in_each_graph = count_triangles(random_graphs, M_k_loops)

In [197]:
M_k = np.mean(n_triangles_in_each_graph)
M_k_var = np.var(n_triangles_in_each_graph)

print("M_k:", M_k)
print("var:", M_k_var)

M_k: 15.0072
var: 52.49928816


### T(k)

In [198]:
random_graphs = (np.random.random((k, math.comb(n, 3))) < p**3).astype(int)
N_triangles = random_graphs.sum(axis=1)   

In [199]:
T_k = np.mean(N_triangles)
T_k_var = np.var(N_triangles)

print("T_k:", T_k)
print("var:", T_k_var)

T_k: 14.9918
var: 13.146972760000002


In [200]:
math.comb(n, 3)*0.5**3

15.0

In [202]:
math.comb(n, 3)*(0.5**3 - 0.5**6)

13.125